# Inspection Benchmarking

In this study we wish to check the AI's ability to check the validity of the data and determine if the experiment has succeeded or not.

We generate 100 figures with synthetic data for each experiment. 100 success and different types of failure.

We report the correctness with 1: only fitting 2: only vision model 3: vision model + fitting

In [ ]:
import leeq
import json
from simulated_setup import * # Change to your customized setup file
import numpy as np
from scipy import optimize as so
from leeq.experiments.builtin import *
import plotly.graph_objects as go
from labchronicle import log_and_record, register_browser_function

from leeq.utils.compatibility import *
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.experiments.builtin import *
from pprint import pprint

In [ ]:

dut_dict = {
    'Q1': {'Active': True, 'Tuneup': False,'FromLog':False, 'Params': configuration_a},
    'Q2': {'Active': True, 'Tuneup': False,'FromLog':False, 'Params': configuration_b}
} 


In [ ]:
duts_dict = {}
for hrid, dd in dut_dict.items():
    if (dd['Active']):
        if (dd['FromLog']):
            dut = TransmonElement.load_from_calibration_log(dd['Params']['hrid'])
        else:
            dut = TransmonElement(name=dd['Params']['hrid'],parameters=dd['Params'])
            
        dut.print_config_info()
        duts_dict[hrid] = dut

dut = None

In [ ]:
num_samples = 10

## GMM

In [ ]:
def run_benchmark(scan_params):
    dut = duts_dict['Q1']  
    res = MeasurementCalibrationMultilevelGMM(dut=dut,**scan_params)
    return extract_results_from_experiment(res)

In [ ]:
success_inspections = {}

for i in range(num_samples):
    np.random.seed(i)

    simulation_setup()
    setup().status().set_param("AIAutoInspectPlots", True)  
    ExperimentManager().status().set_parameter("Plot_Result_In_Jupyter",True)
    
    scan_params = {
        'amp' : 0.1+0.05*np.random.random(1)[0],
        'freq': 9645.5+1*(0.5-np.random.random(1)[0]),
    }
    print(scan_params)
        
    result = run_benchmark(scan_params)

    success_inspections[i] = result
    with open('gmm_success_cases.json', 'w') as f:
        json.dump(success_inspections, f)

In [ ]:
failed_inspections = {}

for i in range(num_samples):
    np.random.seed(i)

    offset = 1000+ 10 * np.random.random(1)[0]+2

    simulation_setup()
    setup().status().set_param("AIAutoInspectPlots", True)  
    ExperimentManager().status().set_parameter("Plot_Result_In_Jupyter",True)
    
    scan_params = {
        'amp' : 0.1+0.05*np.random.random(1)[0],
        'freq': 9645.5-10+1*(0.5-np.random.random(1)[0]),
    }

    result = run_benchmark(scan_params)
    
    failed_inspections[i] = result
    with open('gmm_failed_cases.json', 'w') as f:
        json.dump(failed_inspections, f)

In [ ]:
simulation_setup()
dut = duts_dict['Q1']  
meas = MeasurementCalibrationMultilevelGMM(dut=dut)

In [ ]:
meas.hexbin_iq()